<a href="https://colab.research.google.com/github/engeljason/cloud_colab_processing/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-24 07:48:19--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.22MB/s    in 0.2s    

2021-10-24 07:48:20 (5.22 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
!wget https://raw.githubusercontent.com/engeljason/cloud_colab_processing/main/Resources/schema.sql

--2021-10-24 07:48:20--  https://raw.githubusercontent.com/engeljason/cloud_colab_processing/main/Resources/schema.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 637 [text/plain]
Saving to: ‘schema.sql’

schema.sql          100%[===================>]     637  --.-KB/s    in 0s      

2021-10-24 07:48:20 (28.7 MB/s) - ‘schema.sql’ saved [637/637]



In [4]:
files = ['https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz', 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz']
filenames = ['amazon_reviews_us_Video_Games_v1_00.tsv.gz', 'amazon_reviews_us_Software_v1_00.tsv.gz']
prefixes = ['video_games_', 'software_']
i = 0


In [5]:
!wget -q https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz
!gunzip amazon_reviews_us_Video_Games_v1_00.tsv.gz

In [6]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","postgresql-42.2.9.jar").getOrCreate()

In [7]:
with open('amazon_reviews_us_Video_Games_v1_00.tsv') as file:
  print(file.readlines(5))

['marketplace\tcustomer_id\treview_id\tproduct_id\tproduct_parent\tproduct_title\tproduct_category\tstar_rating\thelpful_votes\ttotal_votes\tvine\tverified_purchase\treview_headline\treview_body\treview_date\n']


In [8]:
from pyspark import SparkFiles
url = files[i]
spark.sparkContext.addFile(url)

In [9]:
df = spark.read.csv(SparkFiles.get(filenames[i]), sep="\t", header=True, inferSchema=True)

In [10]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [11]:
from pyspark.sql.types import DateType
df2 = df.withColumn('review_date' , df['review_date'].cast(DateType()))

In [12]:
df2.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [13]:
review_id_df = df2[['review_id', 'customer_id', 'product_id' , 'product_parent' , 'review_date']]
review_id_df.head(1)

[Row(review_id='RTIS3L2M1F5SM', customer_id=12039526, product_id='B001CXYMFS', product_parent=737716809, review_date=datetime.date(2015, 8, 31))]

In [14]:
products_df = df2[['product_id' , 'product_title']].dropDuplicates()

In [15]:
customers_df = df2.groupBy('customer_id').count().withColumnRenamed('count', 'customer_count')
customers_df.head()

Row(customer_id=48670265, customer_count=1)

In [16]:
vine_df = df2[['review_id' , 'star_rating' , 'helpful_votes' , 'total_votes' , 'vine']]

In [17]:
import psycopg2

try:
  conn = psycopg2.connect(database = 'postgres', user = 'postgres', password = <insert password>, host = 'jasonhwrds.crsbnxfeuino.us-west-1.rds.amazonaws.com', port = '5432')
except:
  print("unable to connect")

cur = conn.cursor()

prefix = prefixes[i]

with open('schema.sql') as file:
  qry = file.read()
  qry = qry.replace('CREATE TABLE ', f'CREATE TABLE {prefix}')
  try:
    cur.execute(qry)
  except:
    print("Create table failed")

conn.commit()
conn.close()
cur.close()

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [18]:
mode="append"
jdbc_url = "jdbc:postgresql://jasonhwrds.crsbnxfeuino.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": <insert password>,
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table
review_id_df.write.jdbc(url=jdbc_url, table=f'{prefix}review_id_table', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table=f'{prefix}products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table=f'{prefix}customers', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table=f'{prefix}vine_table', mode=mode, properties=config)